# 1. Download data

**Execute this step only...**
- if you have not previously downloaded the data provided for the `lnsimulator` GitHub [repository](https://github.com/ferencberes/LNTrafficSimulator)

**OR**
- if you are running this notebook on [Binder](https://mybinder.org/)

In [ ]:
%%bash
wget https://dms.sztaki.hu/~fberes/ln/ln_data_2019-10-29.zip
unzip ln_data_2019-10-29.zip
mv ln_data ..

## 2. Prepare LN data

In order to run the simulation you need to provide LN snapshots as well as information about merchants nodes.

### LN snapshots

The network structure is fed to `lnsimulator` in the form of LN snapshots. Raw JSON files as well as preprocessed payment channel data can be used as input.

#### Load data from JSON file

In [ ]:
from lnsimulator.ln_utils import preprocess_json_file

data_dir = "../ln_data/" # path to the ln_data folder that contains the downloaded data
directed_edges = preprocess_json_file("%s/sample.json" % data_dir)

### Merchants

We provided the list of LN merchants that we used in our experiments. This merchant information was collected in early 2019.

In [ ]:
import pandas as pd
node_meta = pd.read_csv("%s/1ml_meta_data.csv" % data_dir)
providers = list(node_meta["pub_key"])

## 3. Configuration

First we give you the list of main parameters.

| Parameter | Description |
|     :---      |   :---   |
| `amount` |  value of each simulated transaction in satoshis  |
| `count`  | number of random transactions to sample  |
| `epsilon` |  ratio of merchants among transactions endpoints  |
| `drop_disabled` | drop temporarily disabled channels |
| `drop_low_cap` | drop channels with capacity less than `amount` |
| `with_depletion` | the available channel capacity is maintained for both endpoints |

You can initialize the traffic simulator by providing the network structure, merchants information and the former parameters.

In [ ]:
import lnsimulator.simulator.transaction_simulator as ts

amount = 60000
count = 7000
epsilon = 0.8
drop_disabled = True
drop_low_cap = True
with_depletion = True

simulator = ts.TransactionSimulator(directed_edges, providers, amount, count, drop_disabled=drop_disabled, drop_low_cap=drop_low_cap, epsilon=epsilon, with_depletion=with_depletion)

## 4. Estimating daily income and traffic

### i.) Transactions

The simulator samples a set of random transactions (during the initialization) that will be used for the estimation of daily node traffic and routing income. You can access the sampled transactions in the form of a `pandas.DataFrame`.

In [ ]:
transactions = simulator.transactions
print(transactions.head())
print(transactions.shape)

### ii.) Run simulation

In this step the simulator searches for cheapest payment paths from transaction senders to its receivers. Channel capacity changes are well maintained during the simulation. 

In [ ]:
_, _, all_router_fees, _ = simulator.simulate(weight="total_fee", with_node_removals=False)

### iii.) Results

After payment simulation you can export the results as well as calculate traffic and income statistics for LN nodes.

In [ ]:
output_dir = "test"
total_income, total_fee = simulator.export(output_dir)

In [ ]:
total_income.set_index("node").head(10)